# 00_preprocess.ipynb
### Preprocess Gridded Data to find Anomalies 

This notebook preprocess gridded sea surface temperature (SST) data to extract anomalies and saves a land mask.

We use monthly mean SST from the [NOAA Optimum Interpolation Sea Surface Temperature](https://www.ncdc.noaa.gov/oisst/data-access) (OISST v2.1) dataset available on S3 compatible object storage.  This data is measured from a blend of satellite and in-situ observations. We elect to use data from AVHRR-only satellites. This product is available from September 1981 through present on a 1/4º global regular grid.

![preprocess_flow](images/00_preprocess.png)


In [1]:
# Import libraries
import s3fs
import sys
import xarray as xr
import numpy as np

#### Import data
- load the daily OISST dataset and resample to monthly means


In [2]:
endpoint_url = 'https://ncsa.osn.xsede.org'
fs_osn = s3fs.S3FileSystem(anon=True, client_kwargs={'endpoint_url': endpoint_url},) 

path = "Pangeo/pangeo-forge/noaa_oisst/v2.1-avhrr.zarr"
ds = xr.open_zarr(fs_osn.get_mapper(path), consolidated=True).resample(time='MS').mean()
ds

,Array,Chunk
Bytes,1.85 GiB,3.96 MiB
Shape,"(478, 1, 720, 1440)","(1, 1, 720, 1440)"
Count,4524 Tasks,478 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,1.85 GiB,3.96 MiB
Shape,"(478, 1, 720, 1440)","(1, 1, 720, 1440)"
Count,4524 Tasks,478 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


In [4]:
%%time
sst = ds.sst.isel(zlev=0).drop('zlev')
sst.load();

/burg/home/hs3277/.conda/envs/marine-heatwaves/lib/python3.9/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


CPU times: user 2min 51s, sys: 1min 1s, total: 3min 52s
Wall time: 7min 30s


<xarray.DataArray 'sst' (time: 478, lat: 720, lon: 1440)>
array([[[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        ...,
        [-1.3066666, -1.3066666, -1.3073332, ..., -1.3066666,
         -1.3066666, -1.3063333],
        [-1.3096668, -1.31     , -1.31     , ..., -1.3093333,
         -1.3093333, -1.3096668],
        [-1.3119999, -1.3119999, -1.3119999, ..., -1.3119999,
         -1.3119999, -1.3119999]],

       [[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
...
        [-1.7448386, -1.7541934, -1.7564517, ..., -1.767097 ,
         -1.7606453, -1.7477419],
        [-1.748387 , -1.7677422, -1.7748388, ..., -1.7838712,
         -1.7735485, -1.75     ],
        [-1.7999997, -1.7999997, -1.7996773, ..., -1.7996773,
         -1.7999997, -1.7999997]],

       [[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        ...,
        [-1.7235714, -1.7385713, -1.7442856, ..., -1.7535715,
         -1.7442856, -1.7264284],
        [-1.727857 , -1.7592859, -1.7721428, ..., -1.7735716,
         -1.762143 , -1.7292856],
        [-1.8      , -1.8      , -1.7914284, ..., -1.7957141,
         -1.8      , -1.8      ]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 1981-09-01 1981-10-01 ... 2021-06-01
  * lat      (lat) float32 -89.88 -89.62 -89.38 -89.12 ... 89.38 89.62 89.88
  * lon      (lon) float32 0.125 0.375 0.625 0.875 ... 359.1 359.4 359.6 359.9

#### Decompose SST maps into mean, trend, annual, and semi-annual harmonics

In [5]:
# Transform time into decimal year 
dyr = sst.time.dt.year + (sst.time.dt.month-0.5)/12

In [ ]:
# Quick check: global mean SST
sst.mean(('lat','lon')).plot()

#### Decompose SST fields into mean, trend, annual, and semi-annual harmonics


In [ ]:
# Our 6 coefficient model is composed of the mean, trend, annual sine and cosine harmonics, & semi-annual sine and cosine harmonics
model = np.array([np.ones(len(dyr))] + [dyr-np.mean(dyr)] + [np.sin(2*np.pi*dyr)] + [np.cos(2*np.pi*dyr)] + [np.sin(4*np.pi*dyr)] + [np.cos(4*np.pi*dyr)])

# Take the pseudo-inverse of model to 'solve' least-squares problem
pmodel = np.linalg.pinv(model)

# Convert model and pmodel to xaray DataArray
model_da = xr.DataArray(model.T, dims=['time','coeff'], coords={'time':sst.time.values, 'coeff':np.arange(1,7,1)}) 
pmodel_da = xr.DataArray(pmodel.T, dims=['coeff','time'], coords={'coeff':np.arange(1,7,1), 'time':sst.time.values})  

# resulting coefficients of the model
sst_mod = xr.DataArray(pmodel_da.dot(sst), dims=['coeff','lat','lon'], coords={'coeff':np.arange(1,7,1), 'lat':sst.lat.values, 'lon':sst.lon.values})


In [ ]:
# Construct mean, trend, and seasonal cycle
mean = model_da[:,0].dot(sst_mod[0,:,:])
trend = model_da[:,1].dot(sst_mod[1,:,:])
seas = model_da[:,2:].dot(sst_mod[2:,:,:])

# compute anomalies by removing all  the model coefficients 
ssta_notrend = sst-model_da.dot(sst_mod)

#### Save preprocessed data to netcdf

In [ ]:
# xarray Dataset to save
ds_out = xr.Dataset(
    data_vars=dict(
        mean=(['time', 'lat', 'lon'], mean.values),
        trend=(['time', 'lat', 'lon'], trend.values),
        seas=(['time', 'lat', 'lon'], seas.values),
        ssta_notrend=(['time', 'lat', 'lon'], ssta_notrend.values),
        
    ),
    coords=dict(
        lon=ds.lon,
        lat=ds.lat,
        time=ds.time,
    ),
    attrs=dict(description="OISST v2.1 preprocessed for Ocetrac",
              threshold='90th percentile',
              climatology='entire period'),
)
ds_out

In [ ]:
ds_out.to_netcdf('~/marine-heatwaves/data/preprocessed_oisst.nc')